# Sequential Inference

This tutorial discusses particle filters, and sequential Monte Carlo more generally.

These techniques are relevant when performing inference on models where there is a large series of factor statements, some of which can be performed earlier than others. This situation often arises in time series models where the factor statements are the result of incoming observations, but the technique works for *any* probabilistic program.

As a motivating example, consider the following program:

In [12]:
:load Plotting.hs 

import Control.Monad.Bayes.Class
import Control.Monad.Bayes.Sampler
import qualified Graphics.Vega.VegaLite as VL
import IHaskell.Display.Hvega (vlShow)
import Control.Applicative
import qualified Data.Text as T


import Pipes (Producer, (>->))
import qualified Pipes as P
import Pipes.Prelude (unfoldr)
import qualified Pipes.Prelude as P
import Control.Monad.Bayes.Weighted

import Control.Monad.Bayes.Inference.SMC
import Control.Monad.Bayes.Inference.SMC2
import Control.Monad.Bayes.Inference.RMSMC
import Control.Monad.Bayes.Population
import Control.Monad.IO.Class

import Data.Ord
import Data.List
import Control.Monad
import Control.Arrow (first)
import Control.Monad.Bayes.Traced

:e OverloadedStrings


In [13]:
ex :: MonadInfer m => m Bool
ex = replicateM 100 do
    ETC

: 

What distribution does this represent? It is the distribution over all lists of Booleans of length $10$, which places all the weight on the sequence `[True,True,True,True,True,True,True,True,True,True]`.

However, the naive approach to exactly inferring this distribution will not work. Why? Because it first constructs all $2^100$ possible solutions, and then throws away all but `[True,True,True,True,True,True,True,True,True,True]`. 

Now if we look at the structure of the program, it's clear that this is unnecessary. Each time a `condition` statement is made, we should throw away all possibilities with $0$ probability mass. If we do this, the size of the set of possible solutions never explodes.

We can perform this *sequential enumeration" with monad-bayes, as follows:

In [ ]:
TODO

: 

`sis` is an inference method which performs a step of inference at each `factor` statement in the program. In the present case, we have used it in conjunction with exact inference, but the idea generalizes naturally to approximate inference methods.

To motivate this, let's examine a problem for which exact inference is no longer feasible, a non-linear state space model.



In [50]:

variance = 3

-- how to move from one latent state to the next
latentTransition :: MonadSample m => (Double, Double) -> m (Double, Double)
latentTransition (x,y) = 
    liftA2 (,) (normal (x+0.5) variance) (normal (y+0.5) variance)


walk :: MonadSample m => Producer (Double,Double) m r
walk = flip P.unfoldr (0,0) $ \s ->
    Right <$> do
        new <- (latentTransition s)
        return (new, new)

toList :: MonadInfer m => P.Producer a m () -> m [a]
toList prod = P.fold  (\x y -> x <> [y]) [] id (prod >-> P.take 100)


Line 13: Redundant bracket
Found:
do new <- (latentTransition s)
   return (new, new)
Why not:
do new <- latentTransition s
   return (new, new)

In [51]:
(xs, ys,xs',ys') <- unzip4 <$> (sampleIOfixed $ prior observedWalk) -- (sampleIO $ generateData 5)

plotVega 
    (zip (zip (xs <> xs') (ys <> ys')) 
    (replicate 100 (T.pack "Latent") <> replicate 100 (T.pack "Observed")))


In [52]:


conditioning :: (MonadSample m, MonadCond m) => P.Producer ((Double,Double)) m ()
conditioning = 
    P.zip walk (P.each (zip xs' ys')) 
    >-> P.chain (\((x,y), (x',y')) -> factor (normalPdf x variance x' * normalPdf y variance y' ))
    >-> P.map fst 


Line 3: Redundant bracket
Found:
((Double, Double))
Why not:
(Double, Double)

In [59]:
smcRes <- sampleIOfixed $ runPopulation $ smcMultinomial 100 2000 (toList conditioning)
(inferredXs, inferredYs) = unzip $ join $ fmap fst smcRes

join $ fmap fst smcRes :: [(Double, Double)]

In [64]:
plotVega 
    (zip (zip (xs <> inferredXs) (ys <> inferredYs)) 
    (replicate 100 (T.pack "Latent") <> replicate 100 (T.pack "Inferred")))

We can see here that `SMC` has done a pretty good job. The intuition is bear in mind is that it is searching the space of solutions with some backtracking, and moreover, this search results in unbiased sampling from the distribution of interest. 

In [69]:


newtype Point = Point {toPair :: [(Double,Double)]}

instance Num Point where
    Point p1 + Point p2 = Point (zipWith (\(x,y)  (x',y') -> ((x+x')/2,(y+y')/2) ) p1 p2)
    fromInteger (_) = undefined

(infX, infY) = unzip $ (toPair . foldr1 (+)  . fmap Point) [ (x,y) | ((x,y), ld) <- smcRes]


Line 8: Redundant bracket
Found:
(_)
Why not:
_Line 10: Use sum
Found:
foldr1 (+)
Why not:
sum

: 

In [70]:
:t smcRes

smcRes :: [([(Double, Double)], Log Double)]

In [68]:
plotVega 
    (zip (zip (xs <> infX) (ys <> infY)) 
    (replicate 100 (T.pack "Latent") <> replicate 100 (T.pack "Inferred")))




In [41]:
(infX, infY) = unzip $ (toPair . foldr1 (+) . fmap Point) [( ( fmap (\((a,b),(c,d)) -> (a,b)))) $ x | x <- take 10 trSamples]

-- (infX, infY) = unzip $ ( ( fmap (\((a,b),(c,d)) -> (a,b)))) $ fst $ head $ sortOn (Down . snd) smcRes


vlShow $ scatter xs ys infX infY

Line 1: Use sum
Found:
foldr1 (+)
Why not:
sumLine 1: Redundant bracket
Found:
((fmap (\ ((a, b), (c, d)) -> (a, b)))) $ x
Why not:
fmap (\ ((a, b), (c, d)) -> (a, b)) $ xLine 1: Use <$>
Found:
((fmap (\ ((a, b), (c, d)) -> (a, b)))) $ x
Why not:
(\ ((a, b), (c, d)) -> (a, b)) <$> xLine 1: Redundant $
Found:
((fmap (\ ((a, b), (c, d)) -> (a, b)))) $ x
Why not:
((fmap (\ ((a, b), (c, d)) -> (a, b)))) x

To perform approximate inference here, we can represent a distribution as a *population* of weighted samples, and at each factor statement, do a resampling operation to keep the population healthy. This is **Sequential Monte Carlo**. Here it is in code:

The algorithm creates a population of $n$ samples (initially weighted equally) and updates them at each factor statement. The update is defined by `smcMultinomial`, which samples new samples from the empirical distribution defined by the population. More sophisticated resamplers are available, including `smcSystematic` (see TODO)